In [ ]:
! pip install langchain_openai langchain_core -U

# JSON Mode

在普通llm调用中，生成的结果没法直接使用在程序中，因为程序一般需要和明确的数据结构交互，而大模型给出的结果往往是非结构化的。

> 假设我们有一个获取获取某地天气的方法get_weather，当我们询问北京的天气时，需要让大模型从用户意图中提取出北京，再输出get_weather需要的格式，最后调用get_weather方法。

```python
def get_weather(params):
    # 从字典中提取 location 参数
    location = params.get('location')    
    return location + "的天气是晴天"
```

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from google.colab import userdata

import os

os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

def get_weather(params):
    # 从字典中提取 location 参数
    location = params.get('location')    
    return location + "的天气是晴天"

model = ChatOpenAI(
    model="qwen2.5-72b-instruct",
    model_kwargs={ "response_format": { "type": "json_object" } },
)

prompt = ChatPromptTemplate.from_template(
    "获取用户问题中提到的城市"
    '你必须始终输出一个包含‘location’键 JSON 对象。'
    "用户问题：{question}"
)

chain = prompt | model | SimpleJsonOutputParser()

res = chain.invoke({ "question": "北京的天气怎么样" })

print(res)
print(type(res))
print()
print(get_weather(res))
